# CruisePlan Demo 

This notebook demonstrates the complete CruisePlan workflow using Python functions rather than CLI commands. Each section corresponds to a CLI subcommand and shows how to accomplish the same tasks programmatically.

## Workflow Overview

The data preparation phase includes steps 1 and 2, the cruise configuration is steps 3 and 4, and the scheduling is step 5.

1. **Bathymetry**: Get bathymetry data for depth calculations
2. **Pangaea**: Search PANGAEA database for relevant oceanographic datasets
3. **Stations**: Interactive station planning (or programmatic configuration)
4. **Process**: Enrich configuration with depths/coordinates + validate
5. **Schedule**: Generate cruise timeline and outputs

All outputs will be saved to `tests_output/demo/` for easy exploration.

## Setup and Imports

In [1]:
# Core imports
import logging
from pathlib import Path

import xarray as xr

import cruiseplan
from cruiseplan.utils.global_ports import (
    add_custom_port,
    get_available_ports,
    list_ports_in_region,
)
from cruiseplan.utils.yaml_io import save_yaml

# Set up output directory
output_dir = Path('../tests_output/demo')
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Demo outputs will be saved to: {output_dir.absolute()}")

# Configure logging to see what's happening
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

Demo outputs will be saved to: /Users/eddifying/Cloudfree/github/cruiseplan/notebooks/../tests_output/demo


## Phase 1: Data Preparation

### Step 1: Download Bathymetry Data

**CLI Equivalent**: 

```bash
cruiseplan bathymetry --bathy-source etopo2022
```

First, we need bathymetry data for depth calculations and visualization.

In [2]:
# Initialize bathymetry manager with ETOPO2022 data
print("📥 Downloading bathymetry data...")
bathy_result = cruiseplan.bathymetry(bathy_source="etopo2022")
bathy_path = bathy_result.data_file

INFO: =� Downloading etopo2022 bathymetry data to /Users/eddifying/Cloudfree/github/cruiseplan/data/bathymetry


📥 Downloading bathymetry data...
File already exists at /Users/eddifying/Cloudfree/github/cruiseplan/data/bathymetry/ETOPO_2022_v1_60s_N90W180_bed.nc (468.5 MB)


### Step 2: Search PANGAEA Database

**CLI Equivalent** (from root, rather than notebooks/ directory): 

```bash
cruiseplan pangaea "CTD" --lat 50 70 --lon -60 -30 --limit 5 --output-dir tests_output/demo --output demo`
```

Search for relevant oceanographic datasets to inform our cruise planning.

In [3]:
print("🔍 Searching PANGAEA database and downloading station data...")

# Define search parameters
query = "CTD"  # or "CTD temperature North Atlantic"
lat_bounds = [50, 70]  # min_lat, max_lat
lon_bounds = [-60, -30]  # min_lon, max_lon
limit = 5

print(f"   Query: '{query}'")
print("   Geographic bounds: 50°N-70°N, 60°W-30°W")
print(f"   Limit: {limit} datasets")

# Search and download in one step
try:
    pangaea_result = cruiseplan.pangaea(
        query_terms=query,
        lat_bounds=lat_bounds,
        lon_bounds=lon_bounds,
        max_results=limit,
        output_dir=str(output_dir),
        output="demo"
    )

    # Extract data from result
    pangaea_stations = pangaea_result.stations_data
    pangaea_files = pangaea_result.files_created

    if pangaea_files:
        print("✅ PANGAEA processing completed!")
        for file in pangaea_files:
            print(f"   📄 {file.name!s}")
    else:
        print("❌ No datasets found or processing failed")

except Exception as e:
    print(f"❌ PANGAEA processing failed: {e}")

🔍 Searching PANGAEA database and downloading station data...
   Query: 'CTD'
   Geographic bounds: 50°N-70°N, 60°W-30°W
   Limit: 5 datasets


INFO: 🔍 Searching PANGAEA for: 'CTD'
INFO: 📍 Geographic bounds: lat [50, 70], lon [-60, -30]
INFO: Search found 1826 total matches. Retrieving first 5...
INFO: ✅ Found 5 datasets
INFO: 📂 DOI file: /Users/eddifying/Cloudfree/github/cruiseplan/tests_output/demo/demo_dois.txt
INFO: 📂 Stations file: /Users/eddifying/Cloudfree/github/cruiseplan/tests_output/demo/demo_stations.pkl
INFO: ⚙️ Processing 5 DOIs...
INFO: 🕐 Rate limit: 1.0 requests/second
INFO: ✅ PANGAEA processing completed successfully!
INFO: 🚀 Next step: cruiseplan stations -p /Users/eddifying/Cloudfree/github/cruiseplan/tests_output/demo/demo_stations.pkl


✅ PANGAEA processing completed!
   📄 demo_dois.txt
   📄 demo_stations.pkl


## Phase 2: Cruise configuration

### Step 3: Create Station Configuration

**CLI Equivalent**: 
```bash
cruiseplan stations --pangaea-file demo_stations.pkl --lat 50 70 --lon -60 -30`
```

**BUT** for this demo, we'll create a programmatic station configuration rather than using the interactive interface.  Maybe we'll make an interactive demo to show the other options, but the interactive part runs nicely from the command line.

In [ ]:
print("🗺️  Creating station configuration...")

# Create a sample cruise configuration
cruise_config = {
    'cruise_name': 'Demo North Atlantic Survey 2025',
    'default_vessel_speed': 10.0,
    'turnaround_time': 30.0,
    'ctd_descent_rate': 1.0,
    'ctd_ascent_rate': 1.0,
    'calculate_transfer_between_sections': True,
    'calculate_depth_via_bathymetry': True,
    'start_date': '2025-06-01T00:00:00Z',

    # Define stations along a transect
    'waypoints': [
        {
            'name': 'STN_001',
            'latitude': 55.0,
            'longitude': -50.0,
            'operation_type': 'CTD',
            'action': 'profile',
            'comment': 'Continental shelf station'
        },
        {
            'name': 'STN_002',
            'latitude': 57.0,
            'longitude': -45.0,
            'operation_type': 'CTD',
            'action': 'profile',
            'comment': 'Slope station'
        },
        {
            'name': 'STN_003',
            'latitude': 59.0,
            'longitude': -40.0,
            'operation_type': 'CTD',
            'action': 'profile',
            'comment': 'Deep water station'
        },
        {
            'name': 'STN_004',
            'latitude': 61.0,
            'longitude': -35.0,
            'operation_type': 'water_sampling',
            'action': 'sampling',
            'duration': 180.0,
            'comment': 'Water sampling station'
        },
        {
            'name': 'MOOR_001',
            'latitude': 60.5,
            'longitude': -38.0,
            'operation_type': 'mooring',
            'action': 'deployment',
            'duration': 240.0,
            'comment': 'Mooring deployment'
        },
        {
            'name': 'STN_005',
            'latitude': 63.0,
            'longitude': -30.0,
            'operation_type': 'CTD',
            'action': 'profile',
            'comment': 'Northern end station'
        },
        {
            'name': 'STN_006',
            'latitude': 58.0,
            'longitude': -42.0,
            'operation_type': 'CTD',
            'action': 'profile',
            'comment': 'Return transect station'
        }
    ],

    # Add scientific transit
    'transects': [
        {
            'name': 'Demo_ADCP_Survey',
            'operation_type': 'underway',
            'action': 'ADCP',
            'vessel_speed': 5.0,
            'route': [
                {'latitude': 56.0, 'longitude': -48.0},
                {'latitude': 56.0, 'longitude': -38.0}
            ],
            'comment': 'Zonal ADCP survey transect'
        }
    ],

    # Define execution order with leg-based port definitions
    'legs': [
        {
            'name': 'Demo_Survey',
            'departure_port': 'port_st_johns',
            'arrival_port': 'port_st_johns',
            'first_waypoint': 'STN_001',
            'last_waypoint': 'STN_006',
            'activities': ['STN_001', 'STN_002', 'STN_003', 'Demo_ADCP_Survey', 'STN_004', 'MOOR_001', 'STN_005', 'STN_006']
        }
    ]
}

# Save configuration
config_file = output_dir / "demo_cruise.yaml"
save_yaml(cruise_config, config_file)

print("✅ Station configuration created")
print(f"   Configuration saved to: {config_file}")
print(f"   Stations: {len(cruise_config['stations'])}")
print(f"   Transects: {len(cruise_config['transects'])}")
print(f"   Legs: {len(cruise_config['legs'])}")

INFO: Saved configuration to: ../tests_output/demo/demo_cruise.yaml


🗺️  Creating station configuration...
✅ Station configuration created
   Configuration saved to: ../tests_output/demo/demo_cruise.yaml
   Stations: 7
   Transects: 1
   Legs: 1


### Step 3.5: Manually edit configuration

The cruise configuration in `stations.yaml` will have some default values that need to be manually updated.  These include the `departure_port` and `arrival_port` within the cruise leg definition.  Default values are called "port_update", but can be replaced using values from the catalog of ports.

```yaml
legs:
  - name: Interactive_Survey
    departure_port: port_update
    arrival_port: port_update
    first_waypoint: STN_001
    last_waypoint: STN_002
    strategy: sequential
    activities:
      - STN_001
      - STN_002
      - Transit_01
      - Area_01
```

In [8]:
# Using cruiseplan.utils.global_ports with get_available_ports() and list_ports_in_region()

ports = get_available_ports()

# Display them nicely
for port_id, description in ports.items():
   print(f"{port_id}: {description}")

# Or get ports in a specific region
north_atlantic_ports = list_ports_in_region(
      min_lat=50.0, max_lat=70.0,
      min_lon=-30.0, max_lon=20.0
  )

for port_id, port_name in north_atlantic_ports.items():
    print(f"{port_id}: {port_name}")

# If you want to add a custom port for your project
add_custom_port("port_my_station", {
    "name": "My Research Station",
    "display_name": "My Research Station, Location",
    "latitude": 60.0,
    "longitude": -20.0,
    "timezone": "GMT+0",
    "description": "Custom research station for this cruise"
    "operation_type": "port"
})

SyntaxError: invalid syntax (28876062.py, line 26)

### Step 4: Process Configuration

**CLI Equivalent**: `cruiseplan enrich -c demo_cruise.yaml --add-depths --add-coords --expand-sections`

Add computed data like depths, formatted coordinates, and expand CTD sections.

In [9]:
# Verify the function is available
print("Type of cruiseplan.process:", type(cruiseplan.process))
print("Is callable:", callable(cruiseplan.process))
print("🔧 Processing configuration (enrich + validate + map)...")
config_file = output_dir / "demo_cruise.yaml"

try:
    process_result = cruiseplan.process(
        config_file=config_file,
        output_dir=str(output_dir),
        output="demo_cruise",
        format="png",
        bathy_source="etopo2022",
        bathy_dir="../data/bathymetry",  # Go up to project root, then into data
        bathy_stride=10,
        figsize=[12, 8],
        depth_check=True,
        tolerance=10
    )

    # Extract data from result
    cruise_config = process_result.config
    generated_config_files = process_result.files_created

    if cruise_config:
        print("✅ Processing completed successfully!")
        print("   - Configuration enriched")
        print("   - Validation passed")
        for file in generated_config_files:
            print(f"   📄 {file!s}")
    else:
        print("❌ Processing failed")

except Exception as e:
    print(f"❌ Processing failed: {e}")
    import traceback
    traceback.print_exc()

INFO: 🔧 Enriching cruise configuration...
INFO: 🔧 Enriching /Users/eddifying/Cloudfree/github/cruiseplan/tests_output/demo/demo_cruise.yaml
INFO: Saved configuration to: /var/folders/t1/z5bp59k95119nw35yqv699t40000gn/T/tmp8c80heoe.yaml
ERROR: ❌ Enrichment failed
Traceback (most recent call last):
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/__init__.py", line 636, in enrich
    enrich_configuration(
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/processing/enrich.py", line 1349, in enrich_configuration
    config_dict, cruise, enrichment_summary = _load_and_validate_config(
                                              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/processing/enrich.py", line 771, in _load_and_validate_config
    cruise = Cruise(temp_config_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/core/cruise.py", line 113, in __init__
    s

Type of cruiseplan.process: <class 'function'>
Is callable: True
🔧 Processing configuration (enrich + validate + map)...
❌ Processing failed: Unexpected error during enrichment: Activity ID 'STN_001' not found in any Catalog (Waypoints, Transects, Areas).


Traceback (most recent call last):
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/__init__.py", line 636, in enrich
    enrich_configuration(
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/processing/enrich.py", line 1349, in enrich_configuration
    config_dict, cruise, enrichment_summary = _load_and_validate_config(
                                              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/processing/enrich.py", line 771, in _load_and_validate_config
    cruise = Cruise(temp_config_path)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/core/cruise.py", line 113, in __init__
    self._resolve_references()
  File "/Users/eddifying/Cloudfree/github/cruiseplan/cruiseplan/core/cruise.py", line 157, in _resolve_references
    leg.activities = self._resolve_mixed_list(leg.activities)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

## Phase 3: Cruise scheduling + outputs

### Step 5: Generate Schedule

**CLI Equivalent**: `cruiseplan schedule -c demo_cruise_enriched.yaml -o tests_output/demo/ --format all`

Generate the complete cruise timeline and output files.

In [7]:
# Find the enriched config file from the previous step
enriched_config_file = None
for files in generated_config_files:
    if str(files).endswith('enriched.yaml'):
        enriched_config_file = files
        break

if not enriched_config_file:
    enriched_config_file = output_dir / "demo_cruise_enriched.yaml"

print("📅 Generating cruise schedule...")

try:
    # Generate the schedule with multiple output formats using new API
    schedule_result = cruiseplan.schedule(
        config_file=enriched_config_file,
        output_dir="../tests_output/demo",
        output="demo_cruise",
        format="html,csv,netcdf",
        leg=None,
        derive_netcdf=True
    )

    # Extract data from result
    timeline = schedule_result.timeline
    generated_files = schedule_result.files_created

    print("✅ Schedule generated successfully!")
    if timeline:
        print(f"   Timeline object created with {len(timeline)} activities")
        print("   Output files in: ../tests_output/demo/")
        print("   Generated formats: HTML, CSV, NetCDF")
        for file in generated_files:
            print(f"   - {file}")

    else:
        print("⚠️ No timeline generated")

except Exception as e:
    print(f"❌ Schedule generation failed: {e}")
    # Optionally show the full traceback for debugging
    import traceback
    traceback.print_exc()

NameError: name 'generated_config_files' is not defined

In [ ]:
# Find the *.nc file in generated_files list of paths (from schedule result)
schedule_file = None
for file in generated_files:
    if str(file).endswith('.nc'):
        schedule_file = file
        break

if schedule_file:
    ds = xr.open_dataset(schedule_file)
    ds
else:
    print("No NetCDF file found in generated files")

## Workflow Summary

Let's review what we've accomplished and check our output files.

In [ ]:
all_files = pangaea_files + [config_file] + generated_config_files + generated_files

print("✅ CruisePlan Demo Complete!")
print(f"📁 Generated files: {len(all_files)} files in {Path(output_dir).name}/")
for file in all_files:
    print(f"   📄 {file.name}")
